#### <font color='orange'>Webscrapping para Preços de imóveis</font>

##### <font color='white'>

**Explicação do Código**

 <br>

 **Objetivo:** Utilizar técnicas Web Scraping ou Raspagem de dados para automatizar a coleta dados de imóveis e organizá-los em um DataFrame de uma forma eficiente, com o objetivo de analisar e visualizar as informações mais relavantes para nossa análise
 
 <br>

1. **Importar Bibliotecas:**

- **Cloudscraper:** É utilizada para contornar algumas proteções de bots, como aqueles criadois pela Cloudflare, facilitabndo a extração dos dados de alguns sites;
- **Pandas:** para manipulação de DataFrames;
- **Json:** conversão dos dados em JSON.

</font>

In [22]:
import pandas as pd
import json
import cloudscraper
from parsel import Selector

In [23]:

# Função para coletar dados de imóveis
def scrape_olx_houses(pages=100):
    scraper = cloudscraper.create_scraper()
    houses_data_bh = []

    for i in range(1, pages + 1):
        try:
            r = scraper.get(f'https://www.olx.com.br/imoveis/aluguel/estado-mg/belo-horizonte-e-regiao?o={i}')
            response = Selector(text=r.text)
            
            script_data = response.xpath('//script[@id="__NEXT_DATA__"]/text()').get()
            if not script_data:
                print(f"Não foram encontrados dados na página {i}")
                continue
            
            html = json.loads(script_data)
            houses = html.get('props', {}).get('pageProps', {}).get('ads', [])
            
            if not houses:
                print(f"Não foram encontrados dados na página {i}")
                continue
            
            for house in houses:
                house_data = {
                    'Titulo Imovel': house.get('title'),
                    'Preco': house.get('price'),
                    'Preço Antigo': house.get('oldPrice'),
                    'Link do Anuncio': house.get('url'),
                    'Localizacao': house.get('location'),
                    'Detalhes': house.get('locationDetails'),
                    'Adicionais': house.get('properties'),
                }
                houses_data_bh.append(house_data)
        
        except Exception as e:
            print(f"Erro ao processar a página {i}: {e}")

    return houses_data_bh

# Coleta dos dados
houses_data_bh = scrape_olx_houses()

# Salvar os dados coletados em um arquivo JSON
with open('olx_houses_bh.json', 'w', encoding='utf-8') as f:
    json.dump(houses_data_bh, f, ensure_ascii=False, indent=4)

print("Dados salvos para olx_houses_bh.json")

# Carregar os dados do JSON
with open('olx_houses_bh.json', 'r', encoding='utf-8') as f:
    houses_data_bh = json.load(f)

# Criar um DataFrame a partir dos dados
df_bh = pd.DataFrame(houses_data_bh)

df_bh.to_excel('olx_houses_bh.xlsx', sheet_name='Imóveis Belo Horizonte',index=False)
print("Dados exportados para imóveis belo horizonte")

Dados salvos para olx_houses_bh.json
Dados exportados para imóveis belo horizonte


In [24]:
#Criando uma cópia para realizar ajustes

df = df_bh.copy()


In [25]:

# Função para corrigir e converter string JSON 

def correct_and_convert_json(adicional_str):
    if isinstance(adicional_str, list):
        # Se já for uma lista, retorná-la diretamente
        return adicional_str
    if adicional_str is None:
        # Se for None, retorna uma lista vazia
        return []
    try:
        # Tenta converter diretamente se for uma string
        return ast.literal_eval(adicional_str)
    except (ValueError, SyntaxError):
        # Em caso de erro, tenta substituir aspas simples por duplas
        try:
            adicional_str = adicional_str.replace("'", '"')
            return ast.literal_eval(adicional_str)
        except (ValueError, SyntaxError) as e:
            # Se ainda falhar, retorna uma lista vazia
            print(f"Erro ao converter: {adicional_str} -> {e}")
            return []

# Aplicar a função de correção e conversão
df['Adicionais'] = df['Adicionais'].apply(correct_and_convert_json)

# Verificar se a conversão foi bem-sucedida
print("Após a conversão:")
print(df['Adicionais'].head())

# Função para transformar a lista de dicionários em um dicionário de colunas
def extract_additional_details(detail_list):
    columns = {}
    for item in detail_list:
        if isinstance(item, dict):  # Verifica se o item é um dicionário
            columns[item.get('label', '')] = item.get('value', '')
    return columns

# Funcao processar o Dataframe

def process_df(df):
    df['Adicionais'] = df['Adicionais'].apply(correct_and_convert_json)
  
    # Aplicar a função a cada linha do Df Original
    df_additional_details = df['Adicionais'].apply(extract_additional_details)
    df_details = df['Detalhes'].apply(pd.Series)

    # Converter a séria de dicionários em um DF
    df_additional_details = pd.DataFrame(df_additional_details.tolist())

    # Concatenar o DF original com o novo Dataframe com os detalhes adicionais pivotados
    df_final = pd.concat([df.drop(columns =["Detalhes", "Adicionais"]),df_details,df_additional_details], axis = 1)

    # Renomear as colunas do DataFrame final
    df_final.rename(columns={'municipality': 'Municipio', 'neighbourhood': 'Bairro','uf':'UF','ddd':'Código Área'}, inplace=True)

    return df_final

# Processar ambos os DataFrames
df_final = process_df(df_bh)

Após a conversão:
0    [{'name': 'category', 'label': 'Categoria', 'v...
1    [{'name': 'category', 'label': 'Categoria', 'v...
2    [{'name': 'category', 'label': 'Categoria', 'v...
3    [{'name': 'category', 'label': 'Categoria', 'v...
4    [{'name': 'category', 'label': 'Categoria', 'v...
Name: Adicionais, dtype: object


In [26]:
# Concatenar os resultados em um único dataframe

arquivo_final = 'Imóveis_Resumo.xlsx'

# Utilizar o ExcelWriter para múltiplos sheets

with pd.ExcelWriter(arquivo_final) as writer:
    df_final.to_excel(writer, sheet_name = 'Imóveis Belo Horizonte')
     
print('Os resultados foram agrupados com sucesso')

Os resultados foram agrupados com sucesso
